## Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, Input,MaxPooling2D,Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
from tensorflow.keras.models import Sequential

In [4]:
import tensorflow as tf

print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 1


## Image Preprocessing

In [5]:
# Set random seed for reproducibility
tf.random.set_seed(42)

# Define parameters
IMG_SIZE = (180,180)
BATCH_SIZE = 32
EPOCHS = 15
NUM_CLASSES = 5
_input = Input((180,180,3))

In [6]:
# Path to your dataset
DATASET_PATH = r'P:\data sets\Tomato_5_class\Training_set'

# Set up data generators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% for validation
)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',   # since you're using sparse labels
    subset='training',
    shuffle=True
)

# No augmentation for validation (just rescaling)
val_aug = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)


validation_generator = val_aug.flow_from_directory(
    DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='validation',
    shuffle=False
)

# # Validation data generator with augmentation
# validation_generator = train_datagen.flow_from_directory(
#     DATASET_PATH,
#     target_size=IMG_SIZE,
#     batch_size=BATCH_SIZE,
#     class_mode='sparse',
#     subset='validation',
#     shuffle=False
# )

Found 4100 images belonging to 5 classes.
Found 1022 images belonging to 5 classes.


In [7]:
# Test data generator (no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)
TEST_DATASET_PATH = r'P:\data sets\Tomato_5_class\Testing_set'

test_generator = test_datagen.flow_from_directory(
    TEST_DATASET_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse'
)

Found 1275 images belonging to 5 classes.


## VGG16 model 

In [8]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input

# Constants
# IMG_SIZE = IMG_SIZE
# NUM_CLASSES = 5

# Load VGG16 base (exclude top FC layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

# Freeze the convolutional base
base_model.trainable = False


In [9]:
# Custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

# Final model
model = Model(inputs=base_model.input, outputs=predictions)

In [10]:
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the head (initial training)
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=5
)

Epoch 1/5
129/129 [==============================] - 88s 582ms/step - loss: 1.6410 - accuracy: 0.4076 - val_loss: 1.3425 - val_accuracy: 0.5225
Epoch 2/5
129/129 [==============================] - 42s 323ms/step - loss: 1.5539 - accuracy: 0.4605 - val_loss: 1.3266 - val_accuracy: 0.5235
Epoch 3/5
129/129 [==============================] - 45s 346ms/step - loss: 1.4837 - accuracy: 0.4832 - val_loss: 1.3133 - val_accuracy: 0.5235
Epoch 4/5
129/129 [==============================] - 46s 357ms/step - loss: 1.4392 - accuracy: 0.4885 - val_loss: 1.3039 - val_accuracy: 0.5294
Epoch 5/5
129/129 [==============================] - 42s 324ms/step - loss: 1.4122 - accuracy: 0.4973 - val_loss: 1.2953 - val_accuracy: 0.5313


In [11]:
EPOCHS_ADDITIONAL = 7

history_more = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS_ADDITIONAL,
    verbose=1
)

Epoch 1/7
129/129 [==============================] - 38s 292ms/step - loss: 1.3636 - accuracy: 0.5198 - val_loss: 1.2948 - val_accuracy: 0.5323
Epoch 2/7
129/129 [==============================] - 39s 304ms/step - loss: 1.3585 - accuracy: 0.5193 - val_loss: 1.2876 - val_accuracy: 0.5401
Epoch 3/7
129/129 [==============================] - 40s 313ms/step - loss: 1.3456 - accuracy: 0.5249 - val_loss: 1.2873 - val_accuracy: 0.5382
Epoch 4/7
129/129 [==============================] - 41s 314ms/step - loss: 1.3297 - accuracy: 0.5351 - val_loss: 1.2878 - val_accuracy: 0.5431
Epoch 5/7
129/129 [==============================] - 41s 314ms/step - loss: 1.3151 - accuracy: 0.5385 - val_loss: 1.2965 - val_accuracy: 0.5479
Epoch 6/7
129/129 [==============================] - 40s 309ms/step - loss: 1.3083 - accuracy: 0.5424 - val_loss: 1.2906 - val_accuracy: 0.5489
Epoch 7/7
129/129 [==============================] - 41s 316ms/step - loss: 1.3028 - accuracy: 0.5454 - val_loss: 1.2797 - val_accuracy:

In [12]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Unfreeze top layers of the base model
for layer in base_model.layers[-8:]:
    layer.trainable = True

# Recompile with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint("best_tomato_model.h5", save_best_only=True, monitor='val_loss')
]

# Fine-tune the model
history_finetune = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/10
129/129 [==============================] - 44s 324ms/step - loss: 1.1418 - accuracy: 0.5815 - val_loss: 1.0880 - val_accuracy: 0.5959
Epoch 2/10
129/129 [==============================] - 42s 325ms/step - loss: 1.0072 - accuracy: 0.6073 - val_loss: 1.0372 - val_accuracy: 0.6018
Epoch 3/10
129/129 [==============================] - 44s 340ms/step - loss: 0.9518 - accuracy: 0.6198 - val_loss: 0.9808 - val_accuracy: 0.6125
Epoch 4/10
129/129 [==============================] - 44s 342ms/step - loss: 0.9099 - accuracy: 0.6229 - val_loss: 0.9481 - val_accuracy: 0.6233
Epoch 5/10
129/129 [==============================] - 44s 342ms/step - loss: 0.8779 - accuracy: 0.6344 - val_loss: 0.9212 - val_accuracy: 0.6262
Epoch 6/10
129/129 [==============================] - 44s 338ms/step - loss: 0.8474 - accuracy: 0.6461 - val_loss: 0.9459 - val_accuracy: 0.6243
Epoch 7/10
129/129 [==============================] - 44s 340ms/step - loss: 0.8392 - accuracy: 0.6554 - val_loss: 1.0009 - val_ac

## MobileNETV2

In [15]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [16]:
# Load MobileNetV2 base
base_model = MobileNetV2(input_shape=IMG_SIZE + (3,), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model

9406464/9406464 [==============================] - 3s 0us/step


In [17]:
# Build model
inputs = Input(shape=IMG_SIZE + (3,))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
outputs = Dense(5, activation='softmax')(x)
model = Model(inputs, outputs)

# Compile
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [18]:
# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint('mobilenetv2_best_model.h5', save_best_only=True, monitor='val_loss')
]

# Train
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/15
129/129 [==============================] - 37s 266ms/step - loss: 1.5349 - accuracy: 0.4398 - val_loss: 1.3803 - val_accuracy: 0.4638
Epoch 2/15
129/129 [==============================] - 36s 278ms/step - loss: 1.3163 - accuracy: 0.5356 - val_loss: 1.2828 - val_accuracy: 0.5029
Epoch 3/15
129/129 [==============================] - 35s 268ms/step - loss: 1.2250 - accuracy: 0.5580 - val_loss: 1.2276 - val_accuracy: 0.5137
Epoch 4/15
129/129 [==============================] - 34s 261ms/step - loss: 1.1919 - accuracy: 0.5576 - val_loss: 1.2104 - val_accuracy: 0.5049
Epoch 5/15
129/129 [==============================] - 35s 268ms/step - loss: 1.1663 - accuracy: 0.5610 - val_loss: 1.1676 - val_accuracy: 0.5352
Epoch 6/15
129/129 [==============================] - 34s 263ms/step - loss: 1.1293 - accuracy: 0.5685 - val_loss: 1.1394 - val_accuracy: 0.5548
Epoch 7/15
129/129 [==============================] - 34s 262ms/step - loss: 1.0920 - accuracy: 0.5861 - val_loss: 1.1256 - val_ac

In [19]:
# Unfreeze top layers of the base model
base_model.trainable = True

# Optionally: Freeze lower layers (to avoid overfitting and save time)
for layer in base_model.layers[:-50]:
    layer.trainable = False

# Recompile with lower learning rate for fine-tuning
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Continue training (fine-tuning phase)
history_finetune = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=15,
    callbacks=callbacks,  # EarlyStopping and Checkpoint
    verbose=1
)

Epoch 1/15
129/129 [==============================] - 38s 268ms/step - loss: 0.9757 - accuracy: 0.6151 - val_loss: 1.0634 - val_accuracy: 0.5881
Epoch 2/15
129/129 [==============================] - 34s 265ms/step - loss: 0.9132 - accuracy: 0.6393 - val_loss: 1.0209 - val_accuracy: 0.5851
Epoch 3/15
129/129 [==============================] - 35s 268ms/step - loss: 0.8850 - accuracy: 0.6434 - val_loss: 0.9860 - val_accuracy: 0.5822
Epoch 4/15
129/129 [==============================] - 34s 267ms/step - loss: 0.8529 - accuracy: 0.6549 - val_loss: 0.9613 - val_accuracy: 0.6233
Epoch 5/15
129/129 [==============================] - 34s 264ms/step - loss: 0.8350 - accuracy: 0.6695 - val_loss: 0.9838 - val_accuracy: 0.5969
Epoch 6/15
129/129 [==============================] - 34s 261ms/step - loss: 0.8202 - accuracy: 0.6668 - val_loss: 0.9804 - val_accuracy: 0.5832
Epoch 7/15
129/129 [==============================] - 35s 273ms/step - loss: 0.8061 - accuracy: 0.6744 - val_loss: 0.9416 - val_ac